### Import packages / setup

In [ ]:
import datetime
import tweepy
from tweepy import OAuthHandler
import json
import pandas as pd
import csv
import re
import string
import os
import time
import random
import numpy as np
from string import punctuation
from collections import Counter, defaultdict
from pprint import pprint
from operator import itemgetter

In [ ]:
# I've put my API keys in a .py file called API_keys.py
from my_api_keys import bearer_token
tweepyclient = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [ ]:
# Authenticate the Tweepy API
#auth = tweepy.OAuthHandler(api_key,api_key_secret)
#auth.set_access_token(access_token, access_token_secret)
#api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# ACADEMIC ACCOUNT
# Set connection to Tweepy

# From the "project set-up" experience
#api_key = 'e9hKR9ngqwXalms6q4IndwoAz'
#api_key_secret = 'mQzPQWyQvO3ikOqMGjJfKpbFTMcSJKNE9vvywpuSi1Jv0HAjGt'
#bearer_token = 'AAAAAAAAAAAAAAAAAAAAADMxgwEAAAAAcEBEj8u5Qy'  



In [ ]:

def scrapetweetstest(query, start_time, end_time, num_tweets):
    
    tweet_list = []
    
    for response in tweepy.Paginator(tweepyclient.search_all_tweets, 
                                     query = query,
                                     user_fields = ['username', 'public_metrics', 'description', 'location'],
                                     tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                     expansions = 'author_id',
                                     start_time = start_time,
                                     end_time = end_time,
                                  max_results=10):
        tweet_list.append(response)
        print(len(tweet_list))
        
        # Begin scraping the tweets individually:
        #noTweets = 0
        
        



In [ ]:
start_time = "2022-10-31T00:00:00Z"
end_time = "2022-11-01T00:00:00Z"
query = f'#trailrunning OR #running OR #run OR #trail OR #trailrunner OR #trailrun -is:retweet lang:en'
num_tweets=100

scrapetweetstest(query, start_time, end_time, num_tweets)

### start here

In [ ]:
result = []
user_dict = {}
# Loop through each response object
for response in trail_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'user_id': tweet.author_id, 
                       'username': author_info['username'],
                       'follower_count': author_info['followers'],
                       'total_tweets': author_info['tweets'],
                       'description': author_info['description'],
                       'location': author_info['location'],
                       'tweet_id' : tweet.id,
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets_count': tweet.public_metrics['retweet_count'],
                       'replies_count': tweet.public_metrics['reply_count'],
                       'likes_count': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [ ]:
df

In [ ]:
len(df)

### Function for scraping tweets

#### The function scrapes 15k tweets per day and stores in a CSV file

In [ ]:
def scrapetweets(search_words, start_time, end_time, numtweets, numruns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try numruns times

    # Define a pandas dataframe to store the data:
    db_tweets = pd.DataFrame(columns = ['user_id','screen_name','description','location','following_count', 
                                        'followers_count', 'totaltweets',
                                        'date_created', 'tweet_id', 'retweetcount','full_text'])
    
    program_start = time.time()
    for i in range(0, numruns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object, a generator function
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepyclient.search_all_tweets(query=search_words,
                                                end_time=end_time,
                                                start_time=start_time
                              )
        
        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
        
        # Begin scraping the tweets individually:
        noTweets = 0
    
        for tweet in tweet_list:
            userid = tweet.user.id
            username = tweet.user.screen_name
            description = tweet.user.description
            location = tweet.user.location
            following_count = tweet.user.friends_count
            follower_count = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            date_created = tweet.created_at
            tweet_id = tweet.id
            retweetcount = tweet.retweet_count
            full_text = tweet.full_text

            # Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [userid, username, description, location, following_count, follower_count, 
                        totaltweets, date_created, tweet_id, retweetcount, full_text]

            # Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet

            # increase counter - noTweets  
            noTweets += 1
                          
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)

        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time taken for run {} to complete is {} mins'.format(i+1, duration_run))

        time.sleep(0) #15 minute sleep time between runs
        
    # Once all runs have completed, save them to a single csv file:
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.date.today().strftime('%Y%m%d_%H%M%S')
    
    # Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_trailrunningtweets.csv'
    
    # Store dataframe in csv with creation date timestamp
    #db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print("\n")
    print(f'Scraping for {startdate} to {enddate} has completed!')
    print('Total time taken to scrape is {} minutes.'.format(round(program_end - program_start)/60, 2))
    
    return db_tweets

### Need to update with pulling followers for each userid in this list

Client.get_users_followers
https://docs.tweepy.org/en/stable/client.html#tweets


In [ ]:
# Set time, search parameters and run scrapetweets function

start_time = "2022-10-27T12:00:00Z"
end_time = "2022-11-01T12:00:00Z"

search_words = f'#trailrunning OR #running OR #run OR #trail OR #trailrunner OR #trailrun -is:retweet'
numtweets=100
numruns=1

scrapetweets(search_words, start_time, end_time, numtweets, numruns)

### testing adding followers & following to this intermediate dataframe db_tweets

In [ ]:
db_tweets = pd.read_csv("data/20221102_000000_trailrunningtweets.csv")

In [ ]:
db_tweets_unique = db_tweets.drop_duplicates(subset = ["user_id"])

In [ ]:
def get_followers_id(person):
    followersid = []
    count=0
    user=api.get_user(screen_name=person)
    user_id=user.id
    number_of_followers=user.followers_count
    status = tweepy.Cursor(api.get_follower_ids, screen_name=person, tweet_mode="extended").items()
    for i in range(0,number_of_followers):
        follower=next(status)
        followersid.append(follower)
        count += 1
    return followersid

In [ ]:
get_followers_id("37chandler")


In [ ]:
start_time = "2022-10-20T00:00:00Z"
end_time = "2022-11-01T00:00:00Z"
query = f'#trailrunning OR #running OR #run OR #trail OR #trailrunner OR #trailrun -is:retweet'

tweet_list = []

tweets = tweepyclient.search_all_tweets(query=query,
                                   end_time=end_time,
                                   start_time=start_time,
                                   max_results=500)
                       
